# view the data gathered

In [64]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
datapath="C:/Users/dexte/Documents/GitHub/3D-textures/"

def get_friction(trial1_forces,trial2_forces,trial3_forces,mass):
    mass=mass/1000 #convert to kg
    gf_to_N = lambda arr: [f * 0.00981 for f in arr]
    all_forces = np.concatenate([gf_to_N(trial1_forces[10:-5]),gf_to_N(trial2_forces[10:-5]),gf_to_N(trial3_forces[10:-5])]).reshape((3,len(gf_to_N(trial1_forces[10:-5]))))
    avg_force = np.max(all_forces,axis=0)
    error = np.std(avg_force)
    avg_force = np.max(avg_force)
    # Compute normal force (assuming gravity = 9.81 m/s²)
    normal_force = mass * 9.81

    # Return coefficient of friction
    mu = avg_force / normal_force
    return mu, error

## load in

In [65]:
def get_info(file):
    name=file.split("/")[-1]
    meta=name.split("_")
    texture=meta[1]
    mass=float(meta[2].replace(".csv",""))
    df=pd.read_csv(file)
    friction,error=get_friction(df['Trial 1 Force'],df['Trial 2 Force'],df['Trial 3 Force'],mass)
    return df, friction,texture,error



In [66]:
directory=datapath+"Friction/data/"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

d={"Textures":[],"Frictions":[],"Error":[]}

for file in files:
    df,friction,texture,error=get_info(directory+file)
    d["Textures"].append(texture)
    d["Frictions"].append(friction)
    d['Error'].append(error)

d=pd.DataFrame(d)
print(d)

  Textures  Frictions     Error
0   carpet   0.339530  0.043743
1   carpet   0.350816  0.025686
2     cork   0.301500  0.013531
3    Efoam   0.362364  0.024953
4    Gfoam   0.114554  0.032390
5  leather   0.466140  0.045772


## visualise

In [67]:
for file in files:
    df,friction,texture=get_info(directory+file)
    average=np.average(df[['Trial 1 Force','Trial 2 Force','Trial 3 Force']],axis=1)
    plt.plot(average,label=texture)
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

ValueError: too many values to unpack (expected 3)